In [8]:
import torch
import pandas as pd

import sys
import os


notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)

from src.utils.text_tools import keep_necessary_columns_mnr

In [9]:
mnr = pd.read_excel("../data/mnr_20250303.ods")
mnr = keep_necessary_columns_mnr(mnr)

mnr.head()

,REF,AUTR,TITR,REPR
0,MNR00329,RIBERA Jusepe de (d'après),Saint Jérôme pénitent,"figure (saint Jérôme, pénitence, crâne, nudité)"
1,MNR00330,COLLANTES Francisco,Arrivée à Bethléem,"scène biblique (Sainte Famille, arrivée, Bethl..."
2,MNR00331,"LUCAS Y VELAZQUEZ, Eugenio (autrefois LUCAS Y ...",Hommes et femmes espagnols à une tribune,"scène (homme : assis, femme : assis, espagnol,..."
3,MNR00332,THULDEN Théodore van ? Anonyme,"Religieuse avec saint Augustin, sainte Agnès e...","figures (saint François d'Assise, saint August..."
4,MNR00333,REYNOLDS Sir Joshua,Portrait de Sir William Chambers (1723-1796),"portrait (Chambers William, homme, en buste, d..."


In [10]:
from src.Embedding import TextEmbeddingFromPretrained

# embedding = TextEmbeddingFromPretrained(model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1")
embedding = TextEmbeddingFromPretrained(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [11]:
mnr2la = {
    "AUTR": "Hersteller/Künstler/Autor:in",
    "TITR": "Titel",
    "REPR": "Beschreibung",
}

embs = []
for i, row in mnr.iterrows():
    emb = []

    for mnr_col in mnr2la.keys():
        if mnr_col in row and pd.notna(row[mnr_col]):
            emb.append(embedding.get_mean_pooling_embedding((row[mnr_col])))
        else:
            emb.append(torch.zeros(embedding.emb_size))
        
    embs.append(torch.stack(emb, dim=1))
    if i % 100 == 0:
        print(f"Processed {i} rows")

embs = torch.stack(embs, dim=1)
print(embs.shape)

save_dict = {
    "refs": mnr["REF"].tolist(),
    "embeddings": embs.squeeze(0)
}

torch.save(save_dict, "../data/embeddings/mnr_text_minilm_mean_pooling_embeddings.pt")

Processed 0 rows
Processed 100 rows
Processed 200 rows
Processed 300 rows
Processed 400 rows
Processed 500 rows
Processed 600 rows
Processed 700 rows
Processed 800 rows
Processed 900 rows
Processed 1000 rows
Processed 1100 rows
Processed 1200 rows
Processed 1300 rows
Processed 1400 rows
Processed 1500 rows
Processed 1600 rows
Processed 1700 rows
Processed 1800 rows
Processed 1900 rows
Processed 2000 rows
Processed 2100 rows
Processed 2200 rows
Processed 2300 rows
Processed 2400 rows
torch.Size([1, 2456, 3, 384])


In [12]:
# load


save_dict = torch.load("../data/embeddings/mnr_text_minilm_mean_pooling_embeddings.pt")
print(save_dict["refs"][:5])
print(save_dict["embeddings"].shape)

['MNR00329', 'MNR00330', 'MNR00331', 'MNR00332', 'MNR00333']
torch.Size([2456, 3, 384])
